In [16]:
# Importar as bibliotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import plotly.express as px

In [2]:
# URL da página da Wikipédia para os componentes do Ibovespa
wiki_url = 'https://pt.wikipedia.org/wiki/Lista_de_companhias_citadas_no_Ibovespa'
response = requests.get(wiki_url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[0]

In [4]:
# Ler a tabela com pandas
df = pd.read_html(str(table))[0]


C:\Users\aaraujo\AppData\Local\Temp\ipykernel_7512\3111507930.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [6]:
# Selecionar apenas a coluna "Código"
tickers_df = df[['Código']]

In [7]:
# Extrair os tickers como uma lista
tickers = tickers_df['Código'].tolist()
tickers = [ticker + '.SA' for ticker in tickers] # Adiciona o sufixo .SA para tickers brasileiros

In [8]:
print(tickers)

['ABEV3.SA', 'AZUL4.SA', 'B3SA3.SA', 'BBAS3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BBSE3.SA', 'BEEF3.SA', 'BPAC11.SA', 'BRAP4.SA', 'BRFS3.SA', 'BRKM5.SA', 'BRSR6.SA', 'CCRO3.SA', 'CIEL3.SA', 'CMIG4.SA', 'COGN3.SA', 'CPFE3.SA', 'CRFB3.SA', 'CSAN3.SA', 'CSNA3.SA', 'CVCB3.SA', 'CYRE3.SA', 'ECOR3.SA', 'EGIE3.SA', 'ELET3.SA', 'ELET6.SA', 'EMBR3.SA', 'ENGI11.SA', 'EQTL3.SA', 'FLRY3.SA', 'GGBR4.SA', 'GOAU4.SA', 'GOLL4.SA', 'HAPV3.SA', 'HYPE3.SA', 'IRBR3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'KLBN11.SA', 'LREN3.SA', 'MGLU3.SA', 'MRFG3.SA', 'MRVE3.SA', 'MULT3.SA', 'NTCO3.SA', 'PCAR3.SA', 'PETR3.SA', 'PETR4.SA', 'QUAL3.SA', 'RADL3.SA', 'RAIL3.SA', 'RENT3.SA', 'SANB11.SA', 'SBSP3.SA', 'SUZB3.SA', 'TAEE11.SA', 'TOTS3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'WEGE3.SA', 'YDUQ3.SA']


In [10]:
# Buscar os dados de mercado para os tickers
data = yf.download(tickers, period='1d', group_by='ticker')

[*********************100%%**********************]  64 of 64 completed


In [11]:
# Preparar os dados
stocks = []
for ticker in tickers:
  try:
    info = yf.Ticker(ticker).info
    stock = {
    'Symbol': ticker,
    'Name': info.get('shortName', ''),
    'Sector': info.get('sector', ''),
    'Market Cap': info.get('marketCap', 0),
    'Change': (data[ticker]['Close'][0] - data[ticker]['Open'][0]) / data[ticker]['Open'][0] * 100
    }
    stocks.append(stock)
  except Exception as e:
    print(f"Error fetching data for {ticker}: {e}")

C:\Users\aaraujo\AppData\Local\Temp\ipykernel_7512\2902161821.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Change': (data[ticker]['Close'][0] - data[ticker]['Open'][0]) / data[ticker]['Open'][0] * 100


In [12]:
df_stocks = pd.DataFrame(stocks)

In [13]:
# Definir os intervalos e as cores correspondentes
color_group = [-4, -2, -1, 0, 1, 2, 6]
colors = ['red', 'indianred', 'gray', 'lightgreen', 'lime', 'green']

In [14]:
# Categorizar os valores de 'Change' e atribuir cores
df_stocks['Color'] = pd.cut(df_stocks['Change'], bins=color_group, labels=colors, include_lowest=True)

In [15]:
# Garantir que a coluna Color esteja incluída no DataFrame
print(df_stocks[['Symbol', 'Change', 'Color']])

      Symbol    Change       Color
0   ABEV3.SA -0.349040        gray
1   AZUL4.SA  0.948505  lightgreen
2   B3SA3.SA  0.586514  lightgreen
3   BBAS3.SA -0.895863        gray
4   BBDC3.SA -0.089608        gray
..       ...       ...         ...
59  UGPA3.SA  0.232231  lightgreen
60  USIM5.SA -0.127385        gray
61  VALE3.SA -0.797399        gray
62  WEGE3.SA  0.047620  lightgreen
63  YDUQ3.SA  1.127818        lime

[64 rows x 3 columns]


In [17]:
# Criar o treemap
fig = px.treemap(
df_stocks,
path=['Sector', 'Symbol'],
values='Market Cap',
color='Color',
hover_data=['Symbol', 'Change'],
color_discrete_map={
'red': 'red',
'indianred': 'indianred',
'gray': 'gray',
'lightgreen': 'lightgreen',
'lime': 'lime',
'green': 'green'
},
title='Desempenho das Empresas do Ibovespa',
template='plotly_dark'
)

In [19]:
# Adicionar a mudança percentual aos rótulos
fig.data[0].texttemplate = "%{label}<br>%{customdata[1]:.2f}%"

In [22]:
# Atualizar o layout para melhor aparência
fig.update_traces(
textinfo="label+text",
textfont_size=15,
textfont=dict(color='black'),
textposition="middle center"
)

fig.update_layout(
margin=dict(t=50, l=25, r=25, b=25),
font=dict(size=18)
)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'branchvalues': 'total',
              'customdata': array([['ABEV3.SA', -0.3490398055835884, 'gray'],
                                   ['VALE3.SA', -0.7973989786543946, 'gray'],
                                   ['USIM5.SA', -0.12738537791289042, 'gray'],
                                   ...,
                                   ['(?)', '(?)', '(?)'],
                                   ['(?)', '(?)', '(?)'],
                                   ['(?)', '(?)', '(?)']], dtype=object),
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('labels=%{label}<br>Market Cap=' ... '{customdata[2]}<extra></extra>'),
              'ids': array(['Consumer Defensive/ABEV3.SA', 'Basic Materials/VALE3.SA',
                            'Basic Materials/USIM5.SA', 'Financial Services/SANB11.SA',
                            'Healthcare/QUAL3.SA', 'Energy/PETR4.SA', 'Real Estate/MRVE3.SA',
                            'Industrials/ITSA4.SA', 'Financial Services/IRBR3.SA',
                            'Industrials/GOLL4.SA', 'Basic Materials/GOAU4.SA',
                            'Consumer Cyclical/CVCB3.SA', 'Technology/CIEL3.SA',
                            'Basic Materials/GGBR4.SA', 'Financial Services/BBDC4.SA',
                            'Financial Services/BBAS3.SA', 'Financial Services/BBSE3.SA',
                            'Financial Services/BBDC3.SA', 'Financial Services/BRAP4.SA',
                            'Consumer Defensive/BRFS3.SA', 'Consumer Defensive/JBSS3.SA',
                            'Consumer Defensive/MRFG3.SA', 'Industrials/ECOR3.SA',
                            'Consumer Defensive/NTCO3.SA', 'Utilities/EQTL3.SA',
                            'Consumer Cyclical/LREN3.SA', 'Financial Services/BRSR6.SA',
                            'Healthcare/RADL3.SA', 'Industrials/RAIL3.SA', 'Utilities/TAEE11.SA',
                            'Energy/UGPA3.SA', 'Financial Services/B3SA3.SA',
                            'Industrials/AZUL4.SA', 'Energy/PETR3.SA',
                            'Financial Services/BPAC11.SA', 'Financial Services/ITUB4.SA',
                            'Financial Services/HAPV3.SA', 'Consumer Defensive/COGN3.SA',
                            'Utilities/CPFE3.SA', 'Consumer Defensive/CRFB3.SA', 'Energy/CSAN3.SA',
                            'Basic Materials/BRKM5.SA', 'Consumer Cyclical/CYRE3.SA',
                            'Utilities/ELET3.SA', 'Utilities/ELET6.SA', 'Utilities/EGIE3.SA',
                            'Industrials/EMBR3.SA', 'Utilities/ENGI11.SA', 'Industrials/WEGE3.SA',
                            'Consumer Defensive/BEEF3.SA', 'Industrials/CCRO3.SA',
                            'Technology/TOTS3.SA', 'Utilities/SBSP3.SA', 'Real Estate/MULT3.SA',
                            'Basic Materials/KLBN11.SA', 'Basic Materials/CSNA3.SA',
                            'Healthcare/FLRY3.SA', 'Consumer Cyclical/MGLU3.SA',
                            'Healthcare/HYPE3.SA', 'Consumer Cyclical/PCAR3.SA',
                            'Industrials/RENT3.SA', 'Consumer Defensive/YDUQ3.SA',
                            'Basic Materials/SUZB3.SA', 'Utilities/CMIG4.SA', 'Basic Materials',
                            'Consumer Cyclical', 'Consumer Defensive', 'Energy',
                            'Financial Services', 'Healthcare', 'Industrials', 'Real Estate',
                            'Technology', 'Utilities'], dtype=object),
              'labels': array(['ABEV3.SA', 'VALE3.SA', 'USIM5.SA', 'SANB11.SA', 'QUAL3.SA', 'PETR4.SA',
                               'MRVE3.SA', 'ITSA4.SA', 'IRBR3.SA', 'GOLL4.SA', 'GOAU4.SA', 'CVCB3.SA',
                               'CIEL3.SA', 'GGBR4.SA', 'BBDC4.SA', 'BBAS3.SA', 'BBSE3.SA', 'BBDC3.SA',
                               'BRAP4.SA', 'BRFS3.SA', 'JBSS3.SA', 'MRFG3.SA', 'ECOR3.SA', 'NTCO3.SA',
                               'EQTL3.SA', 'LREN3.SA', 'BRSR6.SA', 'RADL3.SA', 'RAIL3.SA', 'TAEE11.SA',
                       

In [1]:
import time

In [2]:
time.time()

1719859600.7652323

In [3]:
from datetime import datetime

In [9]:
datetime.fromtimestamp(1704067220.0949)

datetime.datetime(2023, 12, 31, 21, 0, 20, 94900)